<a href="https://colab.research.google.com/github/felipeocgusmao/Projeto/blob/main/Financial_Control_System_with_Colors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
import json
import os
import sys
import uuid
import copy
import csv
from datetime import datetime, timedelta
import calendar
import glob
import math
from collections import Counter

# --- Constants and Configuration ---
ARQUIVO_DADOS = "dados_financeiros.json"
ARQUIVO_LOCALES = "locales.json"
TEXTS = {}

# --- ANSI Color Codes ---
class Colors:
    """A class to hold ANSI color codes for terminal output."""
    RESET = '\033[0m'
    BOLD = '\033[1m'
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    MAGENTA = '\033[95m'
    CYAN = '\033[96m'
    WHITE = '\033[97m'

# --- Core Functions ---
def load_locale(language_code='pt'):
    """Loads the appropriate language strings from the locales file."""
    global TEXTS
    default_lang = {"menu_title": "MENU", "invalid_option": "Invalid Option"}
    try:
        with open(ARQUIVO_LOCALES, 'r', encoding='utf-8') as f:
            all_locales = json.load(f)
            TEXTS = all_locales.get(language_code, all_locales.get('en', default_lang))
    except (FileNotFoundError, json.JSONDecodeError):
        TEXTS = default_lang

def get_text(key, **kwargs):
    """Gets a localized text string by its key."""
    text = TEXTS.get(key, key.replace('_', ' ').capitalize())
    return text.format(**kwargs)

def clear_screen():
    """Clears the terminal screen."""
    os.system('clear')

def generate_id():
    """Generates a unique ID."""
    return str(uuid.uuid4())

def get_current_month_key():
    """Gets the key for the current month (e.g., '2024-05')."""
    return datetime.now().strftime("%Y-%m")

def format_currency(value, config):
    """Formats a float value into a currency string."""
    symbol = config.get("currency_symbol", "EUR ")
    color = Colors.GREEN if value >= 0 else Colors.RED
    return f"{color}{symbol}{value:.2f}{Colors.RESET}"

def generate_ascii_bar(current, total, width=20):
    """Generates a colored ASCII progress bar."""
    if total <= 0:
        return f"[{Colors.YELLOW}{'-'*width}{Colors.RESET}] (N/A)"

    percentage = current / total

    # Determine color based on percentage
    if percentage > 1.0:
        color = Colors.RED
    elif percentage >= 0.8:
        color = Colors.YELLOW
    else:
        color = Colors.GREEN

    filled_length = int(percentage * width)
    # Ensure the bar doesn't exceed the total width
    filled_length = min(width, max(0, filled_length))

    bar = color + '#' * filled_length + Colors.RESET + '-' * (width - filled_length)
    return f"[{bar}] {color}{percentage*100:.0f}%{Colors.RESET}"

def select_item_from_list(items, prompt, config, show_progress=False, page_size=8):
    """Displays a paginated list of items for user selection."""
    if not items:
        print(f"{Colors.YELLOW}{get_text('no_items_available')}{Colors.RESET}")
        return None

    current_page = 0
    total_pages = math.ceil(len(items) / page_size)

    while True:
        clear_screen()
        print(f"{Colors.BOLD}{Colors.CYAN}{prompt}{Colors.RESET}")

        start_index = current_page * page_size
        end_index = start_index + page_size
        page_items = items[start_index:end_index]

        for i, item in enumerate(page_items):
            item_num = start_index + i + 1
            if show_progress and 'valor_total' in item:
                progress = (item['valor_atual'] / item['valor_total']) * 100 if item['valor_total'] > 0 else 0
                print(f"  {item_num}. {item['nome']} ({format_currency(item['valor_atual'], config)} / {format_currency(item['valor_total'], config)}) - {progress:.1f}%")
            elif 'dia' in item:
                 print(f"  {item_num}. {item['descricao']} - {get_text('day')} {item['dia']}")
            else:
                tipo_char = item.get('tipo', 'Var')[0].upper()
                tipo_color = Colors.MAGENTA if tipo_char == 'F' else Colors.BLUE
                print(f"  {item_num}. [{tipo_color}{tipo_char}{Colors.RESET}] {item['descricao']} - {format_currency(item['valor'], config)}")

        print(f"\n{get_text('page')} {current_page + 1}/{total_pages}")
        nav_prompt = f"{Colors.YELLOW}{get_text('pagination_prompt_full')}{Colors.RESET}"
        escolha_str = input(nav_prompt).lower()

        if escolha_str in ['p', 'n'] and current_page < total_pages - 1:
            current_page += 1
        elif escolha_str in ['a', 'b'] and current_page > 0:
            current_page -= 1
        elif escolha_str in ['s', 'q']:
            return None
        else:
            try:
                escolha_num = int(escolha_str) - 1
                if 0 <= escolha_num < len(items):
                    return items[escolha_num]
            except ValueError:
                print(f"{Colors.RED}{get_text('invalid_input')}{Colors.RESET}")

def carregar_dados():
    """Loads financial data from the JSON file."""
    if not os.path.exists(ARQUIVO_DADOS):
        return {"config": {"currency_symbol": "EUR ", "language": "pt", "last_run_month": None, "last_checked_reminders": None}, "metas_poupanca": [], "dividas": [], "historico_mensal": {}, "lembretes": []}
    with open(ARQUIVO_DADOS, 'r') as f:
        try:
            dados = json.load(f)
            # Ensure all necessary keys exist to prevent crashes
            dados.setdefault("config", {}).setdefault("currency_symbol", "EUR ")
            dados.setdefault("config", {}).setdefault("language", "pt")
            dados.setdefault("config", {}).setdefault("last_run_month", None)
            dados.setdefault("config", {}).setdefault("last_checked_reminders", None)
            dados.setdefault("metas_poupanca", [])
            dados.setdefault("dividas", [])
            dados.setdefault("historico_mensal", {})
            dados.setdefault("lembretes", [])
            return dados
        except (json.JSONDecodeError, TypeError):
            # Return a default structure if the file is corrupted or empty
            return {"config": {"currency_symbol": "EUR ", "language": "pt", "last_run_month": None, "last_checked_reminders": None}, "metas_poupanca": [], "dividas": [], "historico_mensal": {}, "lembretes": []}

def salvar_dados(dados):
    """Saves financial data to the JSON file."""
    with open(ARQUIVO_DADOS, 'w') as f:
        json.dump(dados, f, indent=4)

def manage_settings(config):
    """Manages user settings like language and currency."""
    clear_screen()
    print(f"--- {Colors.BOLD}{Colors.CYAN}{get_text('settings')}{Colors.RESET} ---")
    print(f"1. {get_text('change_language')}")
    print(f"2. {get_text('change_currency')}")
    escolha = input("> ")
    if escolha == '1':
        new_lang = input(f"{get_text('language_prompt')} (pt/en): ").lower()
        if new_lang in ['pt', 'en']:
            config['language'] = new_lang
            load_locale(new_lang)
            print(f"{Colors.GREEN}{get_text('language_updated')}{Colors.RESET}")
    elif escolha == '2':
        new_symbol = input(f"{get_text('currency_prompt')} ({config['currency_symbol'].strip()}): ")
        if new_symbol:
            config['currency_symbol'] = new_symbol.strip() + " "
            print(f"{Colors.GREEN}{get_text('currency_updated')}{Colors.RESET}")

def handle_new_month_logic(todos_os_dados):
    """Handles logic for when a new month begins, like importing fixed expenses."""
    current_month_key = get_current_month_key()
    last_run_month = todos_os_dados["config"].get("last_run_month")

    if current_month_key != last_run_month and last_run_month is not None:
        clear_screen()
        print(f"{Colors.BOLD}{Colors.MAGENTA}{get_text('welcome_back')} {datetime.now().strftime('%B')}! ✨{Colors.RESET}")

        previous_month_data = todos_os_dados["historico_mensal"].get(last_run_month, {})
        despesas_fixas_anteriores = [d for d in previous_month_data.get("despesas", []) if d.get("tipo") == "fixo"]

        if despesas_fixas_anteriores:
            prompt = f"{Colors.YELLOW}{get_text('import_prompt')} {last_run_month}? (s/n): {Colors.RESET}"
            if input(prompt).lower() == 's':
                if current_month_key not in todos_os_dados["historico_mensal"]:
                    todos_os_dados["historico_mensal"][current_month_key] = {"renda": 0.0, "despesas": []}

                # Deepcopy to avoid modifying previous month's data
                todos_os_dados["historico_mensal"][current_month_key]["despesas"] = copy.deepcopy(despesas_fixas_anteriores)
                print(f"{Colors.GREEN}{get_text('import_success')}{Colors.RESET}")
                input(get_text('press_enter'))

    # Update the last run month to the current month
    todos_os_dados["config"]["last_run_month"] = current_month_key

def gerar_insight_financeiro(dados_mes, config):
    """Generates a simple financial insight based on current data."""
    renda = dados_mes.get("renda", 0.0)
    if renda == 0: return None

    despesas = dados_mes.get("despesas", [])
    if not despesas:
        return f"{Colors.YELLOW}{get_text('insight_no_expenses')}{Colors.RESET}"

    today = datetime.now()
    percent_month_passed = today.day / calendar.monthrange(today.year, today.month)[1]
    gastos_atuais = sum(d['valor'] for d in despesas)

    # Projection insight
    if percent_month_passed > 0.1: # Only show after 10% of the month has passed
        projecao_total = gastos_atuais / percent_month_passed
        if projecao_total > renda * 1.05:
            formatted_proj = format_currency(projecao_total, config)
            return f"{Colors.RED}💡 {get_text('insight_projection_alert', valor=formatted_proj)}{Colors.RESET}"

    # Top spending category insight
    gastos_por_categoria = Counter(d['categoria'] for d in despesas)
    if gastos_por_categoria:
        maior_categoria, _ = gastos_por_categoria.most_common(1)[0]
        return f"{Colors.YELLOW}💡 {get_text('insight_main_category', categoria=get_text(maior_categoria))}{Colors.RESET}"

    return None

def verificar_despesas_recorrentes(todos_os_dados, despesa_adicionada):
    """Checks if a new variable expense might be a recurring one."""
    if despesa_adicionada.get("tipo") == "fixo": return

    mes_atual_key = get_current_month_key()
    historico = todos_os_dados["historico_mensal"]
    ocorrencias = 0
    meses_anteriores = sorted([k for k in historico if k < mes_atual_key], reverse=True)[:2]

    for mes in meses_anteriores:
        for despesa_antiga in historico[mes].get("despesas", []):
            # Check for similar description and value
            if (despesa_adicionada['descricao'].lower() in despesa_antiga['descricao'].lower() or
                despesa_antiga['descricao'].lower() in despesa_adicionada['descricao'].lower()):
                if abs(despesa_adicionada['valor'] - despesa_antiga['valor']) < despesa_adicionada['valor'] * 0.1: # 10% tolerance
                    ocorrencias += 1
                    break

    if ocorrencias >= 2:
        print("-" * 20)
        prompt = f"{Colors.YELLOW}💡 {get_text('insight_recurring_expense', desc=despesa_adicionada['descricao'])} (s/n): {Colors.RESET}"
        if input(prompt).lower() == 's':
            despesa_adicionada['tipo'] = 'fixo'
            print(f"{Colors.GREEN}{get_text('expense_marked_fixed')}{Colors.RESET}")

def alocar_poupanca(todos_os_dados, despesa):
    """Allocates a savings expense to a specific goal or debt."""
    print(f"{Colors.CYAN}{get_text('allocate_to')}{Colors.RESET}\n1. {get_text('goal')}\n2. {get_text('debt_payment')}")
    aloc_tipo = input("> ")
    if aloc_tipo == '1':
        meta = select_item_from_list(todos_os_dados["metas_poupanca"], get_text('select_goal'), todos_os_dados['config'], show_progress=True)
        if meta:
            despesa["alocacao"] = meta["id"]
            meta["valor_atual"] += despesa['valor']
    elif aloc_tipo == '2':
        divida = select_item_from_list(todos_os_dados["dividas"], get_text('select_debt'), todos_os_dados['config'], show_progress=True)
        if divida:
            despesa["alocacao"] = divida["id"]
            divida["valor_atual"] -= despesa['valor']

def suggest_category(todos_os_dados, descricao, valor, tipo, dados_mes):
    """Suggests a category for a new expense based on history."""
    historico = todos_os_dados["historico_mensal"]
    palavras_desc = set(descricao.lower().split())
    categoria = None

    # Search previous months for similar descriptions
    for mes in reversed(sorted(historico.keys())):
        for despesa in reversed(historico[mes].get("despesas", [])):
            palavras_hist = set(despesa['descricao'].lower().split())
            if palavras_desc & palavras_hist: # Check for common words
                sugestao = despesa['categoria']
                prompt = f"{Colors.YELLOW}{get_text('suggest_category')} '{get_text(sugestao)}'? (s/n, default: s): {Colors.RESET}"
                if input(prompt).lower() != 'n':
                    categoria = sugestao
                else:
                    categoria = None # Explicitly set to None if user refuses
                break
        if categoria is not None: break

    if categoria is None:
        print(f"{Colors.CYAN}{get_text('which_category')}{Colors.RESET}\n1. {get_text('needs')}\n2. {get_text('wants')}\n3. {get_text('savings')}")
        cat_num = input("> ")
        if cat_num not in ['1', '2', '3']:
            print(f"{Colors.RED}{get_text('invalid_category')}{Colors.RESET}")
            return
        categoria = {"1": "necessidades", "2": "desejos", "3": "poupanca"}[cat_num]

    nova_despesa = {"id": generate_id(), "descricao": descricao, "valor": valor, "categoria": categoria, "tipo": tipo}

    if categoria == "poupanca":
        alocar_poupanca(todos_os_dados, nova_despesa)

    dados_mes.setdefault("despesas", []).append(nova_despesa)
    print(f"{Colors.GREEN}{get_text('expense')} '{descricao}' {get_text('added_to_category')} '{get_text(categoria)}'.{Colors.RESET}")

    if tipo == 'variavel':
        verificar_despesas_recorrentes(todos_os_dados, nova_despesa)


def adicionar_despesa(dados_mes, dados_gerais):
    """Handles the process of adding a new expense."""
    print(f"\n--- {Colors.BOLD}{Colors.CYAN}{get_text('add_expense')}{Colors.RESET} ---")
    tipo_input = input(f"{get_text('expense_type_prompt')}: ")
    tipo = "fixo" if tipo_input == '1' else "variavel"
    descricao = input(f"{get_text('expense_description')}: ")
    try:
        valor = float(input(f"{get_text('expense_value')} ({dados_gerais['config']['currency_symbol']}): "))
        suggest_category(dados_gerais, descricao, valor, tipo, dados_mes)
    except ValueError:
        print(f"{Colors.RED}{get_text('invalid_value')}{Colors.RESET}")

def gerenciar_despesas(dados_mes, todos_os_dados):
    """Allows editing or deleting an existing expense."""
    config = todos_os_dados['config']
    despesa = select_item_from_list(dados_mes.get("despesas", []), f"--- {get_text('manage_expenses')} ---", config)
    if not despesa: return

    print(f"\n{Colors.CYAN}{get_text('what_to_do')}{Colors.RESET}\n1. {get_text('edit_expense')}\n2. {Colors.RED}{get_text('delete_expense')}{Colors.RESET}\n0. {get_text('cancel')}")
    acao = input("> ")
    if acao == '1':
        print(f"\n--- {Colors.BOLD}{get_text('editing_expense')}{Colors.RESET} --- ({get_text('blank_to_keep')})")
        valor_antigo = despesa['valor']
        alocacao_antiga = despesa.get('alocacao')

        nova_desc = input(f"{get_text('new_description')} ({despesa['descricao']}): ")
        if nova_desc: despesa['descricao'] = nova_desc

        novo_valor_str = input(f"{get_text('new_value')} ({despesa['valor']:.2f}): ")
        novo_valor = despesa['valor']
        if novo_valor_str:
            try:
                novo_valor = float(novo_valor_str)
                despesa['valor'] = novo_valor
            except ValueError:
                print(f"{Colors.RED}{get_text('invalid_value_not_changed')}{Colors.RESET}")

        # If value changed, de-allocate from goal/debt and re-allocate
        if valor_antigo != novo_valor and alocacao_antiga:
            for meta in todos_os_dados.get('metas_poupanca', []):
                if meta['id'] == alocacao_antiga:
                    meta['valor_atual'] -= valor_antigo
                    break
            for divida in todos_os_dados.get('dividas', []):
                if divida['id'] == alocacao_antiga:
                    divida['valor_atual'] += valor_antigo
                    break
            despesa.pop('alocacao', None)

        if despesa['categoria'] == 'poupanca':
            alocar_poupanca(todos_os_dados, despesa)

        print(f"{Colors.GREEN}{get_text('expense_updated')}{Colors.RESET}")

    elif acao == '2':
        prompt = f"{Colors.RED}{get_text('are_you_sure_delete')} '{despesa['descricao']}'? (s/n): {Colors.RESET}"
        if input(prompt).lower() == 's':
            # Reverse any allocation if the expense is deleted
            if 'alocacao' in despesa:
                aloc_id = despesa['alocacao']
                valor_despesa = despesa['valor']
                for meta in todos_os_dados.get('metas_poupanca', []):
                    if meta['id'] == aloc_id:
                        meta['valor_atual'] -= valor_despesa
                        break
                for divida in todos_os_dados.get('dividas', []):
                    if divida['id'] == aloc_id:
                        divida['valor_atual'] += valor_despesa
                        break
            dados_mes["despesas"].remove(despesa)
            print(f"{Colors.GREEN}{get_text('expense_deleted')}{Colors.RESET}")

def show_dashboard(dados_mes, todos_os_dados):
    """Displays the main dashboard with financial summaries."""
    config = todos_os_dados['config']
    renda = dados_mes.get("renda", 0.0)
    despesas = dados_mes.get("despesas", [])
    gastos_totais = sum(d['valor'] for d in despesas)
    saldo_atual = renda - gastos_totais

    # --- Comparison with previous month ---
    mes_atual_key = get_current_month_key()
    data_mes_atual = datetime.strptime(mes_atual_key, "%Y-%m")
    mes_anterior_key = (data_mes_atual.replace(day=1) - timedelta(days=1)).strftime("%Y-%m")
    dados_mes_anterior = todos_os_dados['historico_mensal'].get(mes_anterior_key, {})
    gastos_mes_anterior = sum(d['valor'] for d in dados_mes_anterior.get("despesas", []))

    comparativo_str = ""
    if gastos_mes_anterior > 0:
        variacao = ((gastos_totais - gastos_mes_anterior) / gastos_mes_anterior) * 100
        sinal = "+" if variacao >= 0 else ""
        color = Colors.RED if variacao > 0 else Colors.GREEN
        comparativo_str = f"({color}{sinal}{variacao:.0f}% {get_text('vs_last_month')}{Colors.RESET})"

    # --- Display Dashboard ---
    print(f"--- {Colors.BOLD}{Colors.MAGENTA}{get_text('dashboard_title')}{Colors.RESET} ---")
    print(f"{get_text('income')}: {Colors.GREEN}{format_currency(renda, config)}{Colors.RESET} | "
          f"{get_text('expenses')}: {Colors.RED}{format_currency(gastos_totais, config)}{Colors.RESET} {comparativo_str} | "
          f"{get_text('balance')}: {format_currency(saldo_atual, config)}")
    print("-" * 60)

    # --- Category Breakdown ---
    for cat_key, color in [("necessidades", Colors.BLUE), ("desejos", Colors.MAGENTA), ("poupanca", Colors.CYAN)]:
        gastos_cat = sum(d['valor'] for d in despesas if d['categoria'] == cat_key)
        meta_perc = {"necessidades": 0.5, "desejos": 0.3, "poupanca": 0.2}[cat_key]
        meta_valor = renda * meta_perc
        label_key = f"{cat_key}_summary"
        spent_or_saved = get_text('saved') if cat_key == 'poupanca' else get_text('spent')

        print(f"{color}{Colors.BOLD}{get_text(label_key)}{Colors.RESET} (Meta {meta_perc*100:.0f}%):")
        bar_display = generate_ascii_bar(gastos_cat, meta_valor)
        print(f"  {bar_display} | {spent_or_saved}: {format_currency(gastos_cat, config)} {get_text('of')} {format_currency(meta_valor, config)}")

    # --- Financial Insight ---
    insight = gerar_insight_financeiro(dados_mes, config)
    if insight:
        print(insight)
    print("-" * 60)

# --- Other Feature Functions (Goals, Debts, Reports, etc.) ---

def gerenciar_metas_ou_dividas(dados, tipo):
    """Manages savings goals or debts (create, view, delete)."""
    is_meta = tipo == 'meta'
    key = "metas_poupanca" if is_meta else "dividas"
    nome_singular = get_text('goal') if is_meta else get_text('debt')

    while True:
        clear_screen()
        title = f"--- {Colors.BOLD}{get_text('manage_prefix')} {nome_singular}s{Colors.RESET} ---"
        print(title)
        print(f"1. {get_text('create_new')} {nome_singular}")
        print(f"2. {get_text('view_delete')} {nome_singular}s")
        print(f"0. {get_text('back')}")
        escolha = input("> ")

        if escolha == '1':
            nome = input(f"{get_text('name_of_the')} {nome_singular}: ")
            try:
                valor_total = float(input(f"{get_text('total_value')}: "))
                item = {"id": generate_id(), "nome": nome, "valor_total": valor_total, "valor_atual": 0.0 if is_meta else valor_total}
                dados[key].append(item)
                print(f"{Colors.GREEN}{nome_singular.capitalize()} '{nome}' {get_text('created')}.{Colors.RESET}")
            except ValueError:
                print(f"{Colors.RED}{get_text('invalid_value')}{Colors.RESET}")
        elif escolha == '2':
            item_selecionado = select_item_from_list(dados[key], get_text('select_to_delete'), dados['config'], show_progress=True)
            if item_selecionado:
                prompt = f"{Colors.RED}{get_text('are_you_sure_delete')} '{item_selecionado['nome']}'? (s/n): {Colors.RESET}"
                if input(prompt).lower() == 's':
                    dados[key].remove(item_selecionado)
                    print(f"{Colors.GREEN}{nome_singular.capitalize()} {get_text('deleted')}.{Colors.RESET}")
        elif escolha == '0':
            break

        salvar_dados(dados)
        input(get_text("press_enter"))


def relatorio_final(todos_os_dados, mes_key):
    """Displays a detailed report for a given month."""
    clear_screen()
    dados_mes = todos_os_dados['historico_mensal'].get(mes_key, {})
    config = todos_os_dados['config']

    if not dados_mes or dados_mes.get("renda", 0.0) == 0:
        print(f"{Colors.YELLOW}{get_text('no_data_for_report')}{Colors.RESET}")
        return

    print(f"--- {Colors.BOLD}{get_text('report_detailed')} {get_text('for')} {mes_key}{Colors.RESET} ---")

    despesas_por_categoria = {"necessidades": [], "desejos": [], "poupanca": []}
    for d in dados_mes.get("despesas", []):
        if d['categoria'] in despesas_por_categoria:
            despesas_por_categoria[d['categoria']].append(d)

    for cat_key, color in [("necessidades", Colors.BLUE), ("desejos", Colors.MAGENTA), ("poupanca", Colors.CYAN)]:
        print(f"\n--- {color}{Colors.BOLD}{get_text('category')}: {get_text(cat_key).upper()}{Colors.RESET} ---")
        despesas_da_categoria = despesas_por_categoria[cat_key]
        if not despesas_da_categoria:
            print(get_text('no_expenses_in_category'))
        else:
            for despesa in despesas_da_categoria:
                tipo_char = despesa.get('tipo', 'Var')[0].upper()
                tipo_color = Colors.MAGENTA if tipo_char == 'F' else Colors.BLUE
                print(f"  [{tipo_color}{tipo_char}{Colors.RESET}] {despesa['descricao']}: {format_currency(despesa['valor'], config)}")

    print(f"\n\n--- {Colors.BOLD}{get_text('goals_debts_progress')}{Colors.RESET} ---")
    select_item_from_list(todos_os_dados['metas_poupanca'], f"\n{get_text('goals')}:", config, show_progress=True, page_size=999)
    print("-" * 20)
    select_item_from_list(todos_os_dados['dividas'], f"\n{get_text('debts')}:", config, show_progress=True, page_size=999)

def show_spending_projection(dados_mes, config):
    """Shows a projection of spending for the rest of the month."""
    clear_screen()
    print(f"--- {Colors.BOLD}{get_text('projection')}{Colors.RESET} ---")
    renda = dados_mes.get("renda", 0.0)
    if renda == 0:
        print(f"{Colors.YELLOW}{get_text('set_income_first')}{Colors.RESET}")
        return

    today = datetime.now()
    days_in_month = calendar.monthrange(today.year, today.month)[1]
    percent_month_passed = today.day / days_in_month if today.day > 0 else 1

    print(f"{get_text('today_is')} {today.day}/{today.month} ({percent_month_passed*100:.0f}% {get_text('of_month')}).\n")

    gastos_reais = {"necessidades": 0, "desejos": 0, "poupanca": 0}
    for despesa in dados_mes.get("despesas", []):
        if despesa['categoria'] in gastos_reais:
            gastos_reais[despesa["categoria"]] += despesa["valor"]

    for cat, gasto_atual in gastos_reais.items():
        projecao = gasto_atual / percent_month_passed if percent_month_passed > 0 else gasto_atual
        meta_cat = renda * {"necessidades": 0.5, "desejos": 0.3, "poupanca": 0.2}[cat]

        print(f"{Colors.BOLD}{get_text('category')}: {get_text(cat).capitalize()}{Colors.RESET}")
        print(f"  - {get_text('current_spending')}:    {format_currency(gasto_atual, config)}")
        print(f"  - {get_text('final_projection')}: {format_currency(projecao, config)}")

        if projecao > meta_cat:
            print(f"  - {Colors.RED}{get_text('warning_projection_over')} {format_currency(meta_cat, config)}!{Colors.RESET}")
        print("-" * 20)

def run_budget_simulator(todos_os_dados):
    """Runs a 'what if' scenario with a different income."""
    clear_screen()
    config = todos_os_dados['config']
    print(f"--- {Colors.BOLD}{get_text('simulator')}{Colors.RESET} ---")
    print(f"{Colors.YELLOW}{get_text('simulator_disclaimer')}{Colors.RESET}")

    try:
        sim_renda = float(input(f"\n{get_text('simulate_new_income')} ({config['currency_symbol']}): "))
        gastos_fixos_sim = 0

        if input(f"{get_text('add_simulated_fixed')}? (s/n): ").lower() == 's':
            while True:
                gasto_str = input(f"{get_text('fixed_expense_value')} ({get_text('enter_to_stop')}): ")
                if not gasto_str: break
                gastos_fixos_sim += float(gasto_str)

        print(f"\n--- {Colors.BOLD}{get_text('simulation_results')}{Colors.RESET} ---")
        necessidades, desejos, poupanca = sim_renda * 0.5, sim_renda * 0.3, sim_renda * 0.2
        print(f"\n{get_text('with_income_of')}: {format_currency(sim_renda, config)}")
        print(f"{get_text('your_goals_would_be')}:")
        print(f"  - {Colors.BLUE}{get_text('needs')}{Colors.RESET} (50%): {format_currency(necessidades, config)}")
        print(f"  - {Colors.MAGENTA}{get_text('wants')}{Colors.RESET} (30%):      {format_currency(desejos, config)}")
        print(f"  - {Colors.CYAN}{get_text('savings')}{Colors.RESET} (20%):     {format_currency(poupanca, config)}")

        if gastos_fixos_sim > 0:
            print(f"\n{get_text('with_fixed_expenses_of')}: {format_currency(gastos_fixos_sim, config)}")
            saldo_necessidades = necessidades - gastos_fixos_sim
            print(f"  - {get_text('remaining_for_variable_needs')}: {format_currency(saldo_necessidades, config)}")
            if saldo_necessidades < 0:
                print(f"    {Colors.RED}{get_text('warning_fixed_over_50')}{Colors.RESET}")

    except ValueError:
        print(f"\n{Colors.RED}{get_text('invalid_value')}{Colors.RESET}")

def show_trend_analysis(todos_os_dados):
    """Analyzes spending trends over the last 12 months for a category."""
    clear_screen()
    config = todos_os_dados['config']
    print(f"--- {Colors.BOLD}{get_text('trends')}{Colors.RESET} ---")
    print(f"{get_text('which_category_to_analyze')}\n1. {get_text('needs')}\n2. {get_text('wants')}\n3. {get_text('savings')}")
    cat_num = input("> ")
    if cat_num not in ['1', '2', '3']:
        print(f"{Colors.RED}{get_text('invalid_option')}{Colors.RESET}")
        return

    categoria = {"1": "necessidades", "2": "desejos", "3": "poupanca"}[cat_num]
    historico = todos_os_dados["historico_mensal"]
    meses_ordenados = sorted(historico.keys(), reverse=True)[:12]

    if not meses_ordenados:
        print(f"{Colors.YELLOW}{get_text('not_enough_history')}{Colors.RESET}")
        return

    print(f"\n--- {Colors.BOLD}{get_text('spending_evolution_in')} '{get_text(categoria).capitalize()}'{Colors.RESET} ---")
    gastos_mensais = {mes: sum(d['valor'] for d in historico[mes].get("despesas", []) if d.get('categoria') == categoria) for mes in meses_ordenados}
    max_gasto = max(gastos_mensais.values()) if gastos_mensais else 0

    for mes in sorted(gastos_mensais.keys()):
        gasto = gastos_mensais[mes]
        bar = generate_ascii_bar(gasto, max_gasto, 30)
        print(f"{mes}: {format_currency(gasto, config):<20} {bar}")

def import_csv_statement(dados_mes, todos_os_dados):
    """Imports expenses from a CSV file."""
    clear_screen()
    config = todos_os_dados['config']
    print(f"--- {Colors.BOLD}{get_text('import_csv')}{Colors.RESET} ---")
    filename = input(f"{get_text('csv_filename')}: ")
    try:
        with open(filename, mode='r', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            next(reader) # Skip header
            for row in reader:
                try:
                    # Assuming format: Date, Description, Amount
                    _, desc, amount_str = row
                    amount = float(amount_str)
                    if amount >= 0: continue # Skip income/credits

                    clear_screen()
                    print(f"{Colors.YELLOW}{get_text('transaction_found')}:{Colors.RESET} {desc} | {get_text('value')}: {format_currency(abs(amount), config)}")
                    suggest_category(todos_os_dados, desc, abs(amount), "variavel", dados_mes)
                except (ValueError, IndexError):
                    continue
        print(f"\n{Colors.GREEN}{get_text('import_finished')}.{Colors.RESET}")
    except FileNotFoundError:
        print(f"{Colors.RED}{get_text('file_not_found')}{Colors.RESET}")
    except Exception as e:
        print(f"{Colors.RED}{get_text('error_processing_file')}: {e}{Colors.RESET}")

def manage_backups(todos_os_dados):
    """Handles creating and restoring backups."""
    clear_screen()
    print(f"--- {Colors.BOLD}{get_text('backup')}{Colors.RESET} ---")
    print(f"1. {get_text('create_backup')}")
    print(f"2. {get_text('restore_backup')}")
    escolha = input("> ")
    if escolha == '1':
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = f"backup_{timestamp}.json"
        with open(backup_filename, 'w') as f:
            json.dump(todos_os_dados, f, indent=4)
        print(f"\n{Colors.GREEN}{get_text('backup_created_as')} '{backup_filename}'!{Colors.RESET}")
    elif escolha == '2':
        backups = sorted(glob.glob("backup_*.json"), reverse=True)
        if not backups:
            print(f"\n{Colors.YELLOW}{get_text('no_backups_found')}{Colors.RESET}")
            return

        print(f"\n{get_text('select_backup_to_restore')}:")
        for i, filename in enumerate(backups):
            print(f"  {i+1}. {filename}")

        try:
            backup_choice = int(input("> ")) - 1
            if 0 <= backup_choice < len(backups):
                chosen_backup = backups[backup_choice]
                prompt = f"{Colors.RED}{get_text('are_you_sure_restore')} '{chosen_backup}'? {get_text('data_will_be_overwritten')} (s/n): {Colors.RESET}"
                if input(prompt).lower() == 's':
                    with open(chosen_backup, 'r') as f:
                        dados_backup = json.load(f)
                    salvar_dados(dados_backup)
                    print(f"\n{Colors.GREEN}{get_text('restore_complete_restart')}{Colors.RESET}")
                    exit()
            else:
                print(f"{Colors.RED}{get_text('invalid_selection')}{Colors.RESET}")
        except ValueError:
            print(f"{Colors.RED}{get_text('invalid_input')}{Colors.RESET}")

def handle_quick_add(args, todos_os_dados):
    """Handles adding an expense via command-line arguments."""
    mes_key = get_current_month_key()
    if mes_key not in todos_os_dados["historico_mensal"]:
        todos_os_dados["historico_mensal"][mes_key] = {"renda": 0.0, "despesas": []}
    dados_mes = todos_os_dados["historico_mensal"][mes_key]

    try:
        descricao = args[2]
        valor = float(args[3])
        suggest_category(todos_os_dados, descricao, valor, "variavel", dados_mes)
        salvar_dados(todos_os_dados)
    except (IndexError, ValueError):
        print(f"{Colors.RED}{get_text('quick_add_usage')}{Colors.RESET}")

def gerenciar_lembretes(todos_os_dados):
    """Manages payment reminders."""
    clear_screen()
    print(f"--- {Colors.BOLD}{get_text('manage_reminders')}{Colors.RESET} ---")
    print(f"1. {get_text('create_new')} {get_text('reminder')}")
    print(f"2. {get_text('view_delete')} {get_text('reminders')}")
    print(f"0. {get_text('back')}")
    escolha = input("> ")

    if escolha == '1':
        desc = input(f"{get_text('reminder_description')}: ")
        try:
            dia = int(input(f"{get_text('reminder_day')}: "))
            if 1 <= dia <= 31:
                todos_os_dados['lembretes'].append({'id': generate_id(), 'descricao': desc, 'dia': dia})
                print(f"{Colors.GREEN}{get_text('reminder_created')}{Colors.RESET}")
            else:
                print(f"{Colors.RED}{get_text('invalid_day')}{Colors.RESET}")
        except ValueError:
            print(f"{Colors.RED}{get_text('invalid_input')}{Colors.RESET}")
    elif escolha == '2':
        lembrete = select_item_from_list(todos_os_dados.get('lembretes',[]), get_text('select_reminder_to_delete'), todos_os_dados['config'])
        if lembrete and input(f"{Colors.RED}{get_text('are_you_sure_delete')} '{lembrete['descricao']}'? (s/n): {Colors.RESET}").lower() == 's':
            todos_os_dados['lembretes'].remove(lembrete)
            print(f"{Colors.GREEN}{get_text('reminder_deleted')}{Colors.RESET}")

def verificar_lembretes(todos_os_dados):
    """Checks for pending reminders at the start of the application."""
    hoje = datetime.now()
    ultimo_check_str = todos_os_dados['config'].get('last_checked_reminders')

    if ultimo_check_str == hoje.strftime("%Y-%m-%d"): return

    ultimo_check = datetime.strptime(ultimo_check_str, "%Y-%m-%d") if ultimo_check_str else hoje - timedelta(days=32)

    lembretes_pendentes = []
    for lembrete in todos_os_dados.get('lembretes', []):
        dia_lembrete = lembrete['dia']
        # Check if the reminder day has passed since the last check
        if ultimo_check.day < dia_lembrete <= hoje.day and hoje.month == ultimo_check.month:
             lembretes_pendentes.append(lembrete)

    if lembretes_pendentes:
        clear_screen()
        print(f"--- {Colors.YELLOW}{Colors.BOLD}{get_text('pending_reminders')}{Colors.RESET} ---")
        for lembrete in lembretes_pendentes:
            print(f"- {lembrete['descricao']} ({get_text('day')} {lembrete['dia']})")
            if input(f"  {get_text('register_expense_now')} (s/n): ").lower() == 's':
                adicionar_despesa(todos_os_dados['historico_mensal'][get_current_month_key()], todos_os_dados)
        input(get_text('press_enter'))

    todos_os_dados['config']['last_checked_reminders'] = hoje.strftime("%Y-%m-%d")

def analise_por_descricao(todos_os_dados):
    """Analyzes total spending based on a keyword in the description."""
    clear_screen()
    config = todos_os_dados['config']
    print(f"--- {Colors.BOLD}{get_text('analysis_by_description')}{Colors.RESET} ---")
    keyword = input(f"{get_text('keyword_for_analysis')}: ").lower()
    if not keyword: return

    resultados = {}
    for mes, dados_mes in todos_os_dados['historico_mensal'].items():
        gastos_encontrados = [d for d in dados_mes.get('despesas', []) if keyword in d['descricao'].lower()]
        if gastos_encontrados:
            total_mes = sum(d['valor'] for d in gastos_encontrados)
            resultados[mes] = {'total': total_mes, 'count': len(gastos_encontrados)}

    if not resultados:
        print(f"{Colors.YELLOW}{get_text('no_expenses_found_with_keyword', keyword=keyword)}{Colors.RESET}")
        return

    print(f"\n--- {Colors.BOLD}{get_text('analysis_results_for')} '{keyword}'{Colors.RESET} ---")
    for mes in sorted(resultados.keys()):
        info = resultados[mes]
        print(f"{mes}: {format_currency(info['total'], config)} ({info['count']} {get_text('transactions')})")

# --- Main Application Loop ---
def main():
    """The main function that runs the application loop."""
    todos_os_dados = carregar_dados()
    load_locale(todos_os_dados['config'].get('language', 'pt'))

    verificar_lembretes(todos_os_dados)
    handle_new_month_logic(todos_os_dados)

    while True:
        mes_key = get_current_month_key()
        if mes_key not in todos_os_dados["historico_mensal"]:
            todos_os_dados["historico_mensal"][mes_key] = {"renda": 0.0, "despesas": []}
        dados_mes = todos_os_dados["historico_mensal"][mes_key]

        clear_screen()
        show_dashboard(dados_mes, todos_os_dados)

        # --- Main Menu ---
        print(f"\n--- {Colors.BOLD}{Colors.CYAN}{get_text('menu_title')}{Colors.RESET} ---")
        print(f"1. {get_text('add_expense')}")
        print(f"2. {get_text('manage_expenses')}")
        print(f"3. {get_text('set_income')}")
        print(f"4. {get_text('manage_goals')}")
        print(f"5. {get_text('manage_debts')}")
        print(f"--- {Colors.BOLD}{Colors.CYAN}{get_text('analysis_planning')}{Colors.RESET} ---")
        print(f"6. {get_text('report_detailed')}")
        print(f"7. {get_text('projection')}")
        print(f"8. {get_text('simulator')}")
        print(f"9. {get_text('trends')}")
        print(f"10. {get_text('analysis_by_description')}")
        print(f"11. {get_text('manage_reminders')}")
        print(f"--- {Colors.BOLD}{Colors.CYAN}{get_text('system')}{Colors.RESET} ---")
        print(f"12. {get_text('import_csv')}")
        print(f"13. {get_text('backup')}")
        print(f"14. {get_text('settings')}")
        print(f"0. {Colors.YELLOW}{get_text('exit_save')}{Colors.RESET}")

        escolha = input(f"{Colors.BOLD}> {Colors.RESET}")

        action_map = {
            '1': lambda: adicionar_despesa(dados_mes, todos_os_dados),
            '2': lambda: gerenciar_despesas(dados_mes, todos_os_dados),
            '3': lambda: dados_mes.update({'renda': float(input(f"{get_text('income')}: "))}),
            '4': lambda: gerenciar_metas_ou_dividas(todos_os_dados, 'meta'),
            '5': lambda: gerenciar_metas_ou_dividas(todos_os_dados, 'divida'),
            '6': lambda: relatorio_final(todos_os_dados, mes_key),
            '7': lambda: show_spending_projection(dados_mes, todos_os_dados['config']),
            '8': lambda: run_budget_simulator(todos_os_dados),
            '9': lambda: show_trend_analysis(todos_os_dados),
            '10': lambda: analise_por_descricao(todos_os_dados),
            '11': lambda: gerenciar_lembretes(todos_os_dados),
            '12': lambda: import_csv_statement(dados_mes, todos_os_dados),
            '13': lambda: manage_backups(todos_os_dados),
            '14': lambda: manage_settings(todos_os_dados['config']),
        }

        if escolha in action_map:
            try:
                action_map[escolha]()
            except ValueError:
                print(f"{Colors.RED}{get_text('invalid_input')}{Colors.RESET}")
        elif escolha == '0':
            print(get_text("saving_data"))
            salvar_dados(todos_os_dados)
            print(f"{Colors.GREEN}{get_text('goodbye')}{Colors.RESET}")
            break
        else:
            print(f"{Colors.RED}{get_text('invalid_option')}{Colors.RESET}")

        salvar_dados(todos_os_dados)
        input(f"\n{Colors.YELLOW}{get_text('press_enter')}{Colors.RESET}")

if __name__ == "__main__":
    # Check for command-line arguments for quick actions
    if len(sys.argv) > 1 and sys.argv[1] == 'add':
        todos_os_dados = carregar_dados()
        load_locale(todos_os_dados['config'].get('language', 'pt'))
        handle_quick_add(sys.argv, todos_os_dados)
    else:
        main()